In [151]:
import numpy as np
import pandas as pd
import math

In [152]:
class SimilarityMetric:

    def calculateSimilarity(self, u, v, u_mean, v_mean):
        pass

In [153]:
class WeightedCosineSimilarity(SimilarityMetric):

    def __init__(self, weights):
        self.weights = weights

    def calculateSimilarity(self, u, v, u_mean, v_mean):
        u = np.nan_to_num(u, nan=0)
        u_weight_root = u * np.sqrt(self.weights)
        v = np.nan_to_num(v, nan=0)
        v_weight_root = v * np.sqrt(self.weights)

        return np.dot(u_weight_root, v_weight_root) / (np.linalg.norm(u_weight_root) * np.linalg.norm(v_weight_root) + 1e-9)


In [154]:
class WeightedPCCSimilarity(SimilarityMetric):

    def __init__(self, weights):
        self.weights = weights

    def calculateSimilarity(self, u, v, u_mean, v_mean):
        return self.__weighted_pearson_correlation(u, v, u_mean, v_mean)
    
    def __weighted_pearson_correlation(self, x, y, x_mean, y_mean):
        x_df = pd.DataFrame(x, columns = ['a'])
        y_df = pd.DataFrame(y, columns = ['a'])

        x_indices = x_df[x_df['a'].notnull()].index
        y_indices = y_df[y_df['a'].notnull()].index

        indices = x_indices.intersection(y_indices)

        x_reqd = x_df.iloc[indices]
        y_reqd = y_df.iloc[indices]

        w_reqd = pd.DataFrame(self.weights, columns = ['a']).iloc[indices]

        # Compute weighted covariance
        weighted_covariance = sum((w_reqd['a'] * (x_reqd['a'] - x_mean)) * (x_reqd['a'] - y_mean))

        # Compute weighted standard deviations
        weighted_std_x = math.sqrt(sum(w_reqd['a'] * (x_reqd['a'] - x_mean)**2))
        weighted_std_y = math.sqrt(sum(w_reqd['a'] * (y_reqd['a'] - y_mean)**2))

        # Compute weighted Pearson's correlation coefficient
        if weighted_std_x == 0 or weighted_std_y == 0:
            return 0  # Handle division by zero
        else:
            weighted_corr = weighted_covariance / (weighted_std_x * weighted_std_y)
            return weighted_corr

In [155]:
class CollaborativeFiltering:

    def __init__(self, data, metric:SimilarityMetric):
        """
        Initialize CollaborativeFiltering object with user-item rating data.
        
        Parameters:
        - data: DataFrame containing user-item ratings
        """
        self.data = data
        self.similarity_matrix = None
        self.metric = metric
        self.movies = pd.read_csv("movies.csv")
        self.movies.set_index('MovieID', inplace=True)
    
    def calculate_similarity_matrix(self):
        pass

    def predict_ratings(self, user_id):
        pass
    
    def getMovies(self, user_id, movie_id_list):

        recommendations = []

        for i in movie_id_list:

            user = pd.DataFrame(self.data.loc[user_id], index = self.data.columns, columns = ['a'])
            rating = user.loc[i]
            movie = self.movies.loc[i]
            recommendations.append([i, rating, movie['Title'], movie['Genres']])

        return recommendations
    
    def getSimilarityMatrix(self):
        return self.similarity_matrix

In [156]:
class CollaborativeFilteringUserUser(CollaborativeFiltering):
    def __init__(self, data, metric:SimilarityMetric):
        
        super().__init__(data, metric)
        self.means = self.data.mean(axis=1)

    def calculate_similarity_matrix(self):
        
        # user_item_matrix = self.train_data.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)

        n_users = self.data.shape[0]
        similarity_matrix = np.zeros((n_users, n_users))
        for i in range(n_users):
            for j in range(i+1):
                temp = self.metric.calculateSimilarity(self.data.iloc[i, :], self.data.iloc[j, :], self.means[self.data.index[i]], self.means[self.data.index[j]])
                similarity_matrix[i, j] = temp
                similarity_matrix[j, i] = temp
                
        self.similarity_matrix = pd.DataFrame(similarity_matrix, index=self.data.index, columns=self.data.index)

    def predict_ratings(self, user_id):
        """
        Predict ratings for items for a given user.

        Parameters:
        - user_id: ID of the user for whom to predict ratings

        Returns:
        - DataFrame containing predicted ratings for each item
        """

        predicted_ratings = pd.DataFrame(index=self.data.columns, columns=['PredictedRating'])

        for movie_id in predicted_ratings.index:
            numerator = 0
            denominator = 0

            for other_user_id in self.data.index:
                other_user_rating = self.data.loc[other_user_id, movie_id]
                similarity = self.similarity_matrix.loc[user_id , other_user_id]

                if not np.isnan(other_user_rating):
                    numerator += similarity * (other_user_rating)
                    denominator += abs(similarity)

            predicted_ratings.loc[movie_id, 'PredictedRating'] = numerator / (denominator + 1e-9)
            
        return predicted_ratings
    
    

In [157]:
class WeightsProvider:
    
    def getWeightsArray(self):
        pass

In [158]:
class VarianceWeightsProvider(WeightsProvider):
    
    def __init__(self, data):

        temp = data.replace(0, np.NaN)
        weights = np.array(temp.var(axis=0))
        self.weights = np.nan_to_num(weights, nan=0)

        # print(self.weights.shape)
    
    def getWeightsArray(self):
        return self.weights

In [159]:
data=pd.read_csv("EncodedCombined.csv")

user_item_matrix = data.pivot(index='UserID', columns='MovieID', values='Rating')

/tmp/ipykernel_257931/392683434.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("EncodedCombined.csv")


In [160]:
# WeightProvider = IDFWeightsProvider(user_item_matrix)

# metric = WeightedPCCSimilarity(WeightProvider.getWeightsArray())
# cf = CollaborativeFilteringUserUser(user_item_matrix, metric)

# cf.calculate_similarity_matrix()
# cf.getSimilarityMatrix()

In [161]:
# cf.getSimilarityMatrix().to_csv("IDF_Weighted_PCC_collaborative_filtering_similarity_matix.csv")

In [162]:
# prediction_df = cf.predict_ratings(2)

# ids = prediction_df.sort_values(by=['PredictedRating'].tail(10)).index
# movies = cf.getMovies(2, ids)

# for movie in movies:
#     print(f"Title : {movie[2]} , MovieID : {movie[0]} , Rating : {movie[1]} , Genres : {movie[3]}")

In [163]:
WeightProvider = VarianceWeightsProvider(user_item_matrix)

metric = WeightedCosineSimilarity(WeightProvider.getWeightsArray())
cf = CollaborativeFilteringUserUser(user_item_matrix, metric)
cf.calculate_similarity_matrix()

cf.getSimilarityMatrix()

In [ ]:
cf.getSimilarityMatrix().to_csv("Variance_Weighted_Cosine_collaborative_filtering_similarity_matix.csv")

In [ ]:
prediction_df = cf.predict_ratings(2)

ids = prediction_df.sort_values(by=['PredictedRating']).tail(10).index
movies = cf.getMovies(2, ids)

for movie in movies:
    print(f"Title : {movie[2]} , MovieID : {movie[0]}, Genres : {movie[3]}")

KeyboardInterrupt: 